# Interactive Transects

This notebook illustrates how to make transects of tsunami depth over topography that can be interactively updated by selecting the transect.  

For this demo we use the same test data as used in Visualization Tutorial notebook `Crescent_City_Inundation.ipynb`, but it should work also with other combinations of topography and depth data.


### First set up some things needed in notebook....

In [ ]:
%pylab inline
%matplotlib inline

In [ ]:
from __future__ import print_function
import sys, os

In [ ]:
# local module to read an ascii raster file:
import data_tools

# module for making animations:
import JSAnimation
import animation_tools

### Specify location of data files:

If you don't have the data, you can get it via
```
    $ wget http://depts.washington.edu/clawpack/geoclaw/sample_data/CCtimeframes.tar
    $ tar -cf CCtimeframes.tar
```
to create a directory `CCtimeframes`.  Then set `data_dir` appropriately in the cell below.

In [ ]:
data_dir = '/data/CCtimeframes'  # this location should work on Jupyterhub

# check that directory exists, raise an exception if not:
assert os.path.isdir(data_dir), '*** Data directory not found'

### Read in the topography data and define a function to make a contour plot:

In [ ]:
topo_file = os.path.join(data_dir,'topo.asc')
topo_data_dict = data_tools.read_asc_file(topo_file, verbose=True)

topo = topo_data_dict['values']
X = topo_data_dict['X']
Y = topo_data_dict['Y']

### Make `topo_func` function from raster data that can be evaluated anywhere

In [ ]:
from scipy.interpolate import RegularGridInterpolator
topo_func = RegularGridInterpolator((X[0,:], Y[:,0]), topo.T)

### Read in depth data

Define a function that reads in a single frame from the time-dependent solution.  Return the water depth and also `zeta`, which is defined to be the depth on-shore and the water surface elevation (depth + topography) offshore, since this is more interesting to view than the depth and should be continuous at the shoreline (where `topo == 0`).

In [ ]:
def read_frame(frameno, verbose=False):
    time = 2*frameno  # minutes after earthquake
    fname = 'depth_%s.asc' % str(frameno).zfill(4)
    depth_file = os.path.join(data_dir, fname)
    if verbose:
        print("Reading %s at time %s minutes" % (depth_file, time))
        
    depth_data_dict = data_tools.read_asc_file(depth_file, verbose=False)

    depth = depth_data_dict['values']
    zeta = where(topo>0, depth, topo + depth)
    return depth, zeta, time

In [ ]:
depth, zeta, time = read_frame(40, verbose=True)

## Some functions to plot the depth over topography

See the Visualization Tutorial notebook `Crescent_City_Inundation.ipynb` for more information about things done in the next cell.

In [ ]:

def discrete_cmap_1(clines):
    """
    Construct a discrete color map for the regions between the contour lines
    given in clines. Colors go from turqouise through yellow to red.
    Good for flooding depth.
    """
    from numpy import floor, linspace, hstack, ones, zeros
    nlines = len(clines)
    n1 = int(floor((nlines-1)/2.))
    n2 = nlines - 1 - n1
    Green = hstack([linspace(1,1,n1),linspace(1,0,n2)])
    Red = hstack([linspace(0,0.8,n1), ones(n2)])
    Blue = hstack([linspace(1,0.2,n1), zeros(n2)])
    colors = list(zip(Red,Green,Blue))
    return colors

depth_contour_levels = np.arange(0.1,4.5,0.5)  # every 0.5 m up to 4 m
depth_colors = discrete_cmap_1(depth_contour_levels)


def plot_topo_and_depth(zeta, time, ax=None):
    if ax is None:
        fig = figure(figsize=(6,6))
        ax = axes()
    topo_clines = arange(0,20,2)
    ax.contour(X,Y,topo,topo_clines,colors='k')
    ax.contourf(X,Y,zeta, depth_contour_levels, 
             colors = depth_colors, extend='max')
    CClatitude = 41.75  # to rescale longitude
    ax.set_aspect(1. / cos(pi*CClatitude/180.)) 
    ax.ticklabel_format(format='plain',useOffset=False)
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=20 )
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title('%4i minutes after earthquake' % time)


### Plot the depth over topography

Using the function just defined.  Note that since we used `%matplotlib notebook` in the first cell, this is a figure that can be zoomed or panned.  Click the blue `0/1` button at the top to "close" this figure so it is no longer interactive.

In [ ]:
frameno = 30
depth, zeta, time = read_frame(frameno, verbose=True)
plot_topo_and_depth(zeta, time)

In [ ]:
figs = []  # to accumulate figures

for frameno in range(10,41):
    fig = figure(figsize=(6,6))
    ax = axes()
    depth, zeta, time = read_frame(frameno)
    plot_topo_and_depth(zeta, time, ax);
    figs.append(fig)
    close(fig)
    

In [ ]:
import animation_tools

In [ ]:
animation_tools.interact_animate_figs(figs)

In [ ]:
images = animation_tools.make_images(figs, dpi=150)

In [ ]:
animation_tools.JSAnimate_images(images, figsize=(6,6))

## Define class to allow interactive plotting

The class `DrawLine` allows the reader to select a transect and will then interpolate the solution from the specified event to this transect.

We first define a function `topo_func` that allows us to evaluate the topography at any point `(x,y)`.

The function `plot_transect` is then defined to take two points `(x1,y1)` and `(x2,y2)` and interpolate the topography onto a set of 1000 equally spaced points along the transect (straight line connecting the points).  The function also takes another argument, an `depth_func` function that can also be evaluated at any point and is assumed to return a value of `depth` at this point.

### Enable notebook interaction:

(It might be necessary to restart the kernal and do this instead of `%matplotlib inline` to get interact to work.)

In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

class DrawLine:
    def __init__(self, fig,ax1,ax2,depth_func,topo_func,zeta,time):
        self.figure = fig
        self.ax1 = ax1
        self.ax2 = ax2
        self.xprev = 0
        self.yprev = 0
        self.xnew = 0
        self.ynew = 0
        self.press = None
        self.depth_func = depth_func
        self.topo_func = topo_func

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)

    def on_press(self, event):
        self.xprev = event.xdata
        self.yprev = event.ydata
        self.press = 1
        ax1.plot(self.xprev,self.yprev,'bo')
        self.figure.canvas.draw()


    def on_release(self, event):
        self.press = None
        self.xnew = event.xdata
        self.ynew = event.ydata
        self.ax1.cla()  # clear the old transect

        # replot topo and water depth:
        plot_topo_and_depth(zeta, time, ax1)
        
        # add transect plot:
        self.plot_transect()
        
        # plot red line between points selected:
        self.ax1.plot([self.xprev,self.xnew],[self.yprev,self.ynew],'b-o',lw=3)

        self.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.figure.canvas.mpl_disconnect(self.cidpress)
        self.figure.canvas.mpl_disconnect(self.cidrelease)
        
    def plot_transect(self):
        # points on transect:
        xi = linspace(self.xprev, self.xnew, 1000)
        yi = linspace(self.yprev, self.ynew, 1000)
        
        # evaulate topo and zeta on transect:
        Bi = self.topo_func(list(zip(xi,yi)))
        zi = self.depth_func(list(zip(xi,yi)))
                
        # define surface eta as water depth + topography 
        eta = zi+Bi
    
        # Clear axis 2
        self.ax2.cla()
    
        # plot vs. longitude or latitude depending on orientation:
        if (abs(self.xnew-self.xprev) > 0.5*abs(self.ynew-self.yprev)):
            ti = xi
            self.ax2.set_xlim(min(self.xprev,self.xnew),max(self.xprev,self.xnew))
            xtext = 'longitude'
        else:
            ti = yi
            self.ax2.set_xlim(min(self.yprev,self.ynew),max(self.yprev,self.ynew))
            xtext = 'latitude'
            
        BiPos = where(Bi<0, 0., Bi)
        BiNeg = where(Bi>0, 0, Bi)
        
        
        #self.ax2.fill_between(ti, BiPos, eta, color='b')       # flood water
        #self.ax2.fill_between(ti, BiNeg, 0, color=[.7,.7,1])   # original water
        self.ax2.fill_between(ti, -1e6, eta, color='b') 
        self.ax2.fill_between(ti, -1e6, Bi, color=[.5,1,.5])   # topography
        self.ax2.plot(ti, Bi, 'g', lw=1)                       # topography
        
        self.ax2.set_xlabel(xtext)
        self.ax2.set_ylabel('meters')
        self.ax2.set_title('Elevation vs. %s' % xtext)
    
        # choose limits of vertical axis to give nice plots:
        self.ax2.set_ylim(-10, 10)
    
        self.ax2.ticklabel_format(format='plain',useOffset=False)
        self.ax1.set_title('(%8.4f,%8.4f) to (%8.4f,%8.4f)' % (self.xprev,self.yprev,self.xnew,self.ynew))

## Execute the next cell to start the interactive plot...

In [ ]:
depth, zeta, time = read_frame(frameno=40, verbose=True)
depth_func = RegularGridInterpolator((X[0,:], Y[:,0]), depth.T)

gs = gridspec.GridSpec(10, 3)
fig = plt.figure(figsize=(12,10))
ax1 = plt.subplot(gs[0:6,0:])

ax2 = plt.subplot(gs[7:,0:])


plot_topo_and_depth(zeta,time,ax1)
ax1.set_title("Click a point, drag, and release")

ax2 = plt.subplot(gs[7:,0:])

dr =  DrawLine(fig,ax1,ax2,depth_func,topo_func,zeta,time)
dr.connect()

**Note:** To use this, click on a point in the map, drag the mouse, and then release.  The points and transect do not show up until you release.  It would be nice if they did.

## Possible enhancements/extensions:

- Better colormap
- Make animation of transect